# Welcome to the Japan Living Expenses Calculator!

In [2]:
def get_valid_input(prompt, valid_options):
    """Helper function to get and validate user input."""
    while True:
        user_input = input(prompt).lower()
        if user_input in valid_options:
            return user_input
        print(f"Invalid option. Please choose from {', '.join(valid_options)}.")

def get_user_input():
    other_expenses = get_valid_input("1. Do you want to include other yearly expenses? yes/no --> ", {'yes', 'no'})
    accommodation = get_valid_input("2. Choose your accommodation: dormitory/apartment --> ", {'dormitory', 'apartment'})
    transport = get_valid_input("3. Choose your transportation: train/bicycle/walking --> ", {'train', 'bicycle', 'walking'})
    travel_days = 5  # Default value
    if transport == 'train':
        while True:
            try:
                travel_days = int(input("How many days per week do you go to school? "))
                if 0 <= travel_days <= 7:
                    break
                else:
                    print("Invalid number of days. Please enter a number between 0 and 7.")
            except ValueError:
                print("Invalid input. Please enter a valid number.")
    
    water_choice = get_valid_input("4. Choose your drinking water option: tap water/filtered water/bottled water --> ", {'tap water', 'filtered water', 'bottled water'})
    food_choice = get_valid_input("5. Choose your food option: home cooking/eating out/both --> ", {'home cooking', 'eating out', 'both'})
    
    while True:
        try:
            meals_per_day = int(input("How many meals do you eat per day? "))
            if meals_per_day > 0:
                break
            else:
                print("Please enter a valid number greater than 0.")
        except ValueError:
            print("Invalid input. Please enter a number.")
    
    cooking_days = 0
    if food_choice == 'both':
        while True:
            try:
                cooking_days = int(input("How many days per week do you cook at home? "))
                if 0 <= cooking_days <= 7:
                    break
                else:
                    print("Invalid number of cooking days. Please enter a number between 0 and 7.")
            except ValueError:
                print("Invalid input. Please enter a valid number.")

    return other_expenses, accommodation, transport, travel_days, water_choice, food_choice, meals_per_day, cooking_days

def calculate_accommodation(accommodation):
    acc_list = []
    if accommodation == 'dormitory':
        acc_list += ['dormitory_rent', 'dormitory_cleaning_fee']
        return monthly_expenses['dormitory_rent'], one_time_expenses['dormitory_cleaning_fee'], 0, acc_list
    elif accommodation == 'apartment':
        acc_list += ['apartment_rent', 'electricity_gas_bill', 'water_bill', 'ap_int_cost', 'house_insurance', 'guarantor_fee']
        monthly = monthly_expenses['apartment_rent'] + monthly_expenses['electricity_gas_bill'] + monthly_expenses['water_bill']
        one_time = one_time_expenses['ap_int_cost']
        yearly = yearly_expenses['house_insurance'] + yearly_expenses['guarantor_fee']
        return monthly, one_time, yearly, acc_list
    else:
        raise ValueError("Invalid accommodation option.")

def calculate_transport(transport, travel_days):
    tran_list = []
    if transport == 'train':
        tran_list += ['train_fee']
        weekly_travel_cost = per_time_expenses['train_fee'] * travel_days
        return 0, weekly_travel_cost * 52, tran_list
    elif transport == 'walking':
        return 0, 0, tran_list
    elif transport == 'bicycle':
        tran_list += ['bicycle_cost', 'bicycle_insurance']
        one_time = one_time_expenses['bicycle_cost'] + one_time_expenses['bicycle_insurance']
        return one_time, 0, tran_list
    else:
        raise ValueError("Invalid transport option.")      

def calculate_food(food_choice, meals_per_day, cooking_days=0):
    food_list = []
    if food_choice == 'home cooking':
        food_list += ['home cooking']
        return meals_per_day * per_time_expenses['home_cooking'] * 365, food_list
    elif food_choice == 'eating out':
        food_list += ['eating out']
        return meals_per_day * per_time_expenses['eating_out'] * 365, food_list
    elif food_choice == 'both':
        food_list += ['home cooking', 'eating out']
        eating_out_days = 7 - cooking_days
        weekly_home_cooking = meals_per_day * per_time_expenses['home_cooking'] * cooking_days
        weekly_eating_out = meals_per_day * per_time_expenses['eating_out'] * eating_out_days
        return (weekly_home_cooking + weekly_eating_out) * 52, food_list
    else:
        raise ValueError("Invalid food option.")

def calculate_water(water_choice):
    water_list = []
    if water_choice == 'tap water':
        return 0, 0, water_list
    elif water_choice == 'filtered water':
        water_list += ['water_jar', 'water_filter']
        return one_time_expenses['water_jar'], yearly_expenses['water_filter'], water_list
    elif water_choice == 'bottled water':
        water_list += ['drinking_water']
        return 0, per_time_expenses['drinking_water'] * 365, water_list
    else:
        raise ValueError("Invalid water option.")

def calculate_living_expenses():
    other_expenses, accommodation, transport, travel_days, water_choice, food_choice, meals_per_day, cooking_days = get_user_input()
    
    total_monthly = sum(basic_monthly_expenses.values())
    total_one_time = 0
    if other_expenses == 'yes':
        total_yearly = sum(other_yearly_expenses.values())
    else:
        total_yearly = 0
    
    try:
        # Calculate expenses
        monthly_accommodation, one_time_accommodation, yearly_accommodation, acc_list = calculate_accommodation(accommodation)
        total_monthly += monthly_accommodation
        total_one_time += one_time_accommodation
        total_yearly += yearly_accommodation
        acc_expense = (monthly_accommodation*12) + one_time_accommodation + yearly_accommodation
        
        print(f"\n--Your accomodation expenses--")
        print(acc_list)
        print(f"per year (first year): {acc_expense} yen/yr")
        print(f"per year (next year): {acc_expense-one_time_accommodation} yen/yr")
        
        one_time_transport, yearly_transport, tran_list = calculate_transport(transport, travel_days)
        total_yearly += yearly_transport
        total_one_time += one_time_transport
        
        print(f"\n--Your trainsportation expenses--")
        print(tran_list)
        print(f"per year (first year): {yearly_transport+one_time_transport} yen/yr")
        print(f"per year (next year): {yearly_transport} yen/yr")
        
        food_expense, food_list = calculate_food(food_choice, meals_per_day, cooking_days)
        
        print(f"\n--Your food expenses--")
        print(food_list)
        print(f"per year: {food_expense} yen/yr")
        
        one_time_water, yearly_water, water_list = calculate_water(water_choice)
        total_one_time += one_time_water
        total_yearly += yearly_water
        
        print(f"\n--Your water expenses--")
        print(water_list)
        print(f"per year (first year): {one_time_water+yearly_water} yen/yr")
        print(f"per year (next year): {yearly_water} yen/yr")
        
        # Final calculations
        total_annual_expense_first = (total_monthly * 12) + total_yearly + total_one_time + food_expense
        total_annual_expense_next = total_annual_expense_first - total_one_time

        print(f"\n--The estimated living expenses-- \n1. First year:")
        print(f"  per year: {total_annual_expense_first} yen/yr ({total_annual_expense_first * jpy_to_thb} baht/yr)")
        print(f"  per month: {total_annual_expense_first//12} yen/mo ({(total_annual_expense_first * jpy_to_thb)//12} baht/mo)")
        print(f"2. Next year:")
        print(f"  per year: {total_annual_expense_next} yen/yr ({total_annual_expense_next * jpy_to_thb} baht/yr)")
        print(f"  per month: {total_annual_expense_next//12} yen/mo ({(total_annual_expense_next * jpy_to_thb)//12} baht/mo)")
    
    except ValueError as e:
        print(e)

## Note:
International student dormitory: https://www.osaka-u.ac.jp/en/international/inbound/support/residence.html  
Apartment initial cost: https://e-housing.jp/post/essential-fees-to-budget-for-when-renting-an-apartment-in-japan  
NHI: https://www15.j-server.com/LUCTOSHIMA/ns/tl.cgi/https%3a//www.city.toshima.lg.jp/info/en/insurance.html?SLANG=ja&TLANG=en&XMODE=0&XCHARSET=utf-8&XJSID=0

## Fixed cost:

In [3]:
jpy_to_thb = 0.23

basic_monthly_expenses = {
    'mobile_network': 1000, 
    'nhi': 2600,                               # National Health Insurance (period of stay in Japan > 3 months)
    'room_internet': 0
}

apartment_initial_setup_cost = {
    'guarantor_fee': 21000,
    'agency_fee': 44000,               
    'key_money': 50000,
    'house_insurance': 17000,                  # Every 2 years
    'furniture': 100000                        # Chair, Table, ฺBedding set, Rice cooker, IH stove, Kitchen ware, Fan
}

monthly_expenses = {
    'dormitory_rent': 25200,                   # Tsukumodai Dormitory
    'apartment_rent': 45000,
    'electricity_gas_bill': 5000,
    'water_bill': 2000,
}

one_time_expenses = {
    'dormitory_cleaning_fee': 5200,            # Tsukumodai Dormitory
    'bicycle_cost': 25000,
    'bicycle_insurance': 3000,
    'ap_int_cost': sum(apartment_initial_setup_cost.values()),
    'water_jar': 3000
}

yearly_expenses = {
    'house_insurance': 5000,                   # 10000 every 2 years --> 5000 yearly
    'water_filter': 5000,
    'guarantor_fee': 10000                    
}

per_time_expenses = {
    'train_fee': 170,                          # Tsukumodai Dorm > Kita-senri > (walk to) OU
    'home_cooking': 500,
    'eating_out': 1000,
    'drinking_water': 80,
}

other_yearly_expenses = {
    'tuition_fee': 0,
    'traveling': 40000,                        # Lap trip (20000), personal trip (5000*4) --> 40000
    'clothes': 10000,
    'bathroom_amenity': 9900,                  # Shampoo (1000), Conditioner (1000), Soap (600), Toothbrush (300), Toothpaste (400) --> 3300*3=9900 
}

## Run the calculator

In [4]:
calculate_living_expenses()

1. Do you want to include other yearly expenses? yes/no --> yes
2. Choose your accommodation: dormitory/apartment --> apartment
3. Choose your transportation: train/bicycle/walking --> bicycle
4. Choose your drinking water option: tap water/filtered water/bottled water --> filtered water
5. Choose your food option: home cooking/eating out/both --> both
How many meals do you eat per day? 2
How many days per week do you cook at home? 4

--Your accomodation expenses--
['apartment_rent', 'electricity_gas_bill', 'water_bill', 'ap_int_cost', 'house_insurance', 'guarantor_fee']
per year (first year): 871000 yen/yr
per year (next year): 639000 yen/yr

--Your trainsportation expenses--
['bicycle_cost', 'bicycle_insurance']
per year (first year): 28000 yen/yr
per year (next year): 0 yen/yr

--Your food expenses--
['home cooking', 'eating out']
per year: 520000 yen/yr

--Your water expenses--
['water_jar', 'water_filter']
per year (first year): 8000 yen/yr
per year (next year): 5000 yen/yr

--The

In [5]:
calculate_living_expenses()

1. Do you want to include other yearly expenses? yes/no --> yes
2. Choose your accommodation: dormitory/apartment --> apartment
3. Choose your transportation: train/bicycle/walking --> train
How many days per week do you go to school? 4
4. Choose your drinking water option: tap water/filtered water/bottled water --> tap water
5. Choose your food option: home cooking/eating out/both --> both
How many meals do you eat per day? 2
How many days per week do you cook at home? 3

--Your accomodation expenses--
['apartment_rent', 'electricity_gas_bill', 'water_bill', 'ap_int_cost', 'house_insurance', 'guarantor_fee']
per year (first year): 871000 yen/yr
per year (next year): 639000 yen/yr

--Your trainsportation expenses--
['train_fee']
per year (first year): 35360 yen/yr
per year (next year): 35360 yen/yr

--Your food expenses--
['home cooking', 'eating out']
per year: 572000 yen/yr

--Your water expenses--
[]
per year (first year): 0 yen/yr
per year (next year): 0 yen/yr

--The estimated liv